In [319]:
import xgboost as xgb
import catboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from bayes_opt import BayesianOptimization
import numpy as np

In [320]:
path = "/Users/taehoon/Documents/dacon_movies/data/data"
route = path+"/preprocessed_data/preprocessed_"
train, test = pd.read_csv(route+"train.csv", index_col=0), pd.read_csv(route+"test.csv", index_col=0)


In [321]:
def label_encoding(train, test):
    all_df = pd.concat([train, test], ignore_index=True)
    label_encoder = {}
    str_columns = [(i, j) for i, j in zip(train.dtypes, train.columns)]
    for tup in str_columns:
        dtype, column = tup
        if dtype != "object":
            continue
        val2idx = {i:j for j, i in enumerate(all_df[column].unique())}
        label_encoder[column] = val2idx
        train[column] = train[column].map(val2idx)
        test[column] = test[column].map(val2idx)
    return train, test, label_encoder
train, test, _ = label_encoding(train, test)

In [322]:
train_x, test_x, train_y, test_y = train_test_split(train.drop("box_off_num",axis=1),train["box_off_num"])

def cbt_reg(n_estimators, depth, learning_rate, subsample, l2_leaf_reg):
    params = {
        "n_estimators":int(n_estimators),
        "depth":int(depth) ,
        "learning_rate":learning_rate ,
        "subsample":subsample ,
        "l2_leaf_reg":l2_leaf_reg ,
        }
    cbtr_model = catboost.CatBoostRegressor(
                             **params,
                             bootstrap_type='Bernoulli',
                             eval_metric='RMSE',
                             od_type='Iter',
                             allow_writing_files=False)
    cbtr_model.fit(train_x, train_y, silent=True)
    y_pred = cbtr_model.predict(test_x)
    rmse = mean_squared_error(test_y, y_pred, squared=False)
    r2 = r2_score(test_y, y_pred)
    return 1-rmse

In [323]:
pbounds = {"n_estimators": (150,400),
           "depth": (2,7),
           "learning_rate": (.01, 0.2),
           "subsample":(0.6, 1.),
           "l2_leaf_reg":(0,10),
}
bo = BayesianOptimization(f=cbt_reg, pbounds=pbounds, verbose=2, random_state=42)
bo.maximize(init_points=2, n_iter=50, acq='ei', xi=0.01)
high_score = bo.max
1-cbt_reg(**high_score["params"])

|   iter    |  target   |   depth   | l2_lea... | learni... | n_esti... | subsample |
-------------------------------------------------------------------------------------
| 1         | -1.641e+0 | 3.873     | 9.507     | 0.1491    | 299.7     | 0.6624    |
| 2         | -1.727e+0 | 2.78      | 0.5808    | 0.1746    | 300.3     | 0.8832    |
| 3         | -1.552e+0 | 3.936     | 6.911     | 0.05903   | 358.0     | 0.8439    |
| 4         | -1.445e+0 | 4.532     | 7.151     | 0.02087   | 358.0     | 0.6805    |
| 5         | -1.477e+0 | 4.86      | 7.283     | 0.01      | 358.1     | 0.6       |
| 6         | -1.541e+0 | 4.236     | 7.795     | 0.06002   | 358.6     | 0.9848    |
| 7         | -1.555e+0 | 4.541     | 7.067     | 0.1024    | 357.6     | 0.8049    |
| 8         | -1.473e+0 | 4.553     | 7.088     | 0.01      | 358.3     | 0.6       |
| 9         | -1.55e+06 | 2.377     | 8.119     | 0.09867   | 226.4     | 0.7195    |
| 10        | -1.54e+06 | 4.179     | 7.639     | 0.14

1408791.8124822623